In [1]:
import mne
from bids import BIDSLayout
from PreProcess.utils.utils import LAB_root
from PreProcess.timefreq.multitaper import spectrogram
from PreProcess.navigate import crop_data, channel_outlier_marker
import numpy as np
import matplotlib as mpl
try:
    mpl.use("TkAgg")
except ImportError:
    pass
bids_root = mne.datasets.epilepsy_ecog.data_path()
layout = BIDSLayout(LAB_root + "/BIDS-1.0_SentenceRep/BIDS")
filt = mne.io.read_raw_fif(layout.root + "/derivatives/sub-D00" + str(29) + "_" + "SentenceRep" + "_filt_ieeg.fif")

Opening raw data file C:\Users\ae166\Box\CoganLab\BIDS-1.0_SentenceRep\BIDS/derivatives/sub-D0029_SentenceRep_filt_ieeg.fif...
    Range : 0 ... 3805183 =      0.000 ...  1858.000 secs
Ready.
Opening raw data file C:\Users\ae166\Box\CoganLab\BIDS-1.0_SentenceRep\BIDS\derivatives\sub-D0029_SentenceRep_filt_ieeg-1.fif...
    Range : 3805184 ... 7610367 =   1858.000 ...  3716.000 secs
Ready.
Opening raw data file C:\Users\ae166\Box\CoganLab\BIDS-1.0_SentenceRep\BIDS\derivatives\sub-D0029_SentenceRep_filt_ieeg-2.fif...
    Range : 7610368 ... 9328639 =   3716.000 ...  4555.000 secs
Ready.


In [2]:
# Crop raw data to minimize processing time
new = crop_data(filt)

# Mark channel outliers as bad
channel_outlier_marker(new)

# Exclude bad channels
good = new.copy().drop_channels(new.info['bads'])
good.load_data()

# CAR
good.set_eeg_reference(ref_channels="average", ch_type='seeg')

# Remove intermediates from mem
del new

Reading 0 ... 3824503  =      0.000 ...  1867.433 secs...
Applying average reference.
Applying a custom ('sEEG',) reference.


In [3]:
# fix SentenceRep events
is_sent = False
annot = None
for event in good.annotations:
    if event['description'] in ['Audio']:
        if event['duration'] > 1:
            is_sent = True
        else:
            is_sent = False
    if event['description'] not in [ 'Listen', ':=:' ]:
        if is_sent:
            trial_type = "Sentence/"
        else:
            trial_type = "Word/"
    else:
        trial_type = "Start/"
    event['description'] = trial_type + event['description']
    if annot is None:
        annot = mne.Annotations(**event)
    else:
        event.pop('orig_time')
        annot.append(**event)
good.set_annotations(annot)
# events, event_id = mne.events_from_annotations(good)
# events = mne.merge_events(events, [1, 2], 14, replace_events=True)
# event_id['Start'] = 14
# event_id = dict([(value, key) for key, value in event_id.items()])
# annot = mne.annotations_from_events(events, filt.info['sfreq'], event_id)
# filt.set_annotations(annot)

Used Annotations descriptions: ['Sentence/Audio', 'Sentence/BAD boundary', 'Sentence/EDGE boundary', 'Sentence/Response', 'Sentence/Speak', 'Start/:=:', 'Start/Listen', 'Word/Audio', 'Word/BAD boundary', 'Word/EDGE boundary', 'Word/Mime', 'Word/Response', 'Word/Speak']


In [5]:
freqs = np.arange(10, 200., 2.)
spectra = spectrogram(good, 'Word/Audio', -0.5, 1.5, 'Start', -0.5, 0, freqs, n_jobs=-1, verbose=10)
spectra.plot([57], vmin=0.7, vmax=1.4)

Used Annotations descriptions: ['Sentence/Audio', 'Sentence/BAD boundary', 'Sentence/EDGE boundary', 'Sentence/Response', 'Sentence/Speak', 'Start/:=:', 'Start/Listen', 'Word/Audio', 'Word/BAD boundary', 'Word/EDGE boundary', 'Word/Mime', 'Word/Response', 'Word/Speak']
Not setting metadata
180 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 180 events and 6145 original time points ...
0 bad epochs dropped
Not setting metadata
270 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 270 events and 3073 original time points ...
0 bad epochs dropped


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.2min


KeyboardInterrupt: 